In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


In [2]:
sys.meta_path.append(NotebookFinder())

import argparse
import pandas as pd
import mlp
import data

#parser = argparse.ArgumentParser()

EPOCH = 100
BATCH = 32
HOURS = 1
NEURONS = [9,6]
LR = 1e-2

try:

    filename = "dataset.csv"
    dataset = pd.read_csv(filename, header=0, index_col=0)

    processor = data.DataProcessor(dataset)
    processor.shift(HOURS)

    x_data, y_data = processor.to_numpy_arrays(HOURS)
    processor.build_dataset(x_data, y_data)

    ml = mlp.KerasDenseMLP(processor,NEURONS,LR,HOURS,EPOCH,BATCH)
    
    try:
        ml.model.load_weights(mld.checkpoint)

        ml.evaluate()
        ml.predict()
    except Exception as error:
        print("Error trying to load checkpoint.")
        print(error)
        ml.train()
        ml.evaluate()
        ml.predict()

except ValueError as e:
    print ('Error: ' + e.message)


importing Jupyter notebook from mlp.ipynb
importing Jupyter notebook from data.ipynb
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 9)                 90        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 60        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 159
Trainable params: 159
Non-trainable params: 0
_________________________________________________________________
None
Error trying to load checkpoint.
name 'mld' is not defined
Train on 440 samples, validate on 110 samples
Epoch 1/100
440/440 [==============================] - 0s 424us/step - loss: 58.1

440/440 [==============================] - 0s 40us/step - loss: 1.7456 - mean_absolute_error: 1.0544 - val_loss: 1.2763 - val_mean_absolute_error: 0.8348
Epoch 44/100
440/440 [==============================] - 0s 33us/step - loss: 1.6004 - mean_absolute_error: 1.0353 - val_loss: 3.1075 - val_mean_absolute_error: 1.4454
Epoch 45/100
440/440 [==============================] - 0s 35us/step - loss: 1.6249 - mean_absolute_error: 1.0246 - val_loss: 1.2455 - val_mean_absolute_error: 0.8225
Epoch 46/100
440/440 [==============================] - 0s 41us/step - loss: 1.6419 - mean_absolute_error: 1.0168 - val_loss: 2.2306 - val_mean_absolute_error: 1.2216
Epoch 47/100
440/440 [==============================] - 0s 43us/step - loss: 1.5817 - mean_absolute_error: 1.0206 - val_loss: 1.2645 - val_mean_absolute_error: 0.8293
Epoch 48/100
440/440 [==============================] - 0s 44us/step - loss: 1.5957 - mean_absolute_error: 0.9957 - val_loss: 1.2129 - val_mean_absolute_error: 0.8126
Epoch 49/10

440/440 [==============================] - 0s 40us/step - loss: 1.2705 - mean_absolute_error: 0.8741 - val_loss: 1.2277 - val_mean_absolute_error: 0.8299
Epoch 93/100
440/440 [==============================] - 0s 37us/step - loss: 1.3603 - mean_absolute_error: 0.9158 - val_loss: 1.4241 - val_mean_absolute_error: 0.9067
Epoch 94/100
440/440 [==============================] - 0s 31us/step - loss: 1.2442 - mean_absolute_error: 0.8586 - val_loss: 1.5357 - val_mean_absolute_error: 0.9638
Epoch 95/100
440/440 [==============================] - 0s 40us/step - loss: 1.3116 - mean_absolute_error: 0.8916 - val_loss: 1.3182 - val_mean_absolute_error: 0.8825
Epoch 96/100
440/440 [==============================] - 0s 33us/step - loss: 1.3456 - mean_absolute_error: 0.9104 - val_loss: 1.1773 - val_mean_absolute_error: 0.8091
Epoch 97/100
440/440 [==============================] - 0s 45us/step - loss: 1.3281 - mean_absolute_error: 0.9029 - val_loss: 1.8874 - val_mean_absolute_error: 1.0972
Epoch 98/10

<Figure size 640x480 with 1 Axes>

194/194 [==============================] - 0s 20us/step
loss: 1.190e+00
mean_absolute_error: 8.645e-01


<Figure size 640x480 with 1 Axes>